# Download Hugging Face Models for Air-Gapped Systems <a href="https://colab.research.google.com/github/mostly-ai/mostlyai/blob/main/docs/tutorials/model-download/model-download.ipynb" target="_blank"><img src="https://img.shields.io/badge/Open%20in-Colab-blue?logo=google-colab" alt="Run on Colab"></a>

This notebook helps you download Hugging Face models for use in air-gapped systems. It will:
1. Check which models are already in your cache
2. Download any missing models
3. Show you where the cached models are stored

The downloaded models can then be copied to air-gapped systems.

In [ ]:
%pip install -U huggingface_hub

In [2]:
# List of models to download
# Add or remove models as needed
MODELS_TO_DOWNLOAD = [
    # "Qwen/Qwen2.5-1.5B",           # Open model
    # "meta-llama/Llama-3.2-1B",     # Gated model requiring HF_TOKEN
    # "microsoft/phi-1_5",           # Open model
    # "Qwen/Qwen2.5-0.5B",           # Open model
    "amd/AMD-Llama-135m",  # Open model
    "HuggingFaceTB/SmolLM-135M",  # Open model
]

In [3]:
import os
from pathlib import Path

# using HF_TOKEN environment variable if set
MY_HF_TOKEN = os.environ.get("HF_TOKEN", None)
# or alternatively, you can set the token directly:
# MY_HF_TOKEN = "hf_..."

# using HF_HOME environment variable if set
# MY_HF_HOME = os.environ.get("HF_HOME", None)
# or alternatively, you can set the home directory directly:
# MY_HF_HOME = "/path/to/your/cache"
# os.environ["HF_HOME"] = MY_HF_HOME
# or alternatively, you can set the home directory to the default cache directory:
# MY_HF_HOME = Path.home() / ".cache" / "huggingface"
MY_HF_HOME = Path.home() / "MY_HF_HOME_TEST"
os.environ["HF_HOME"] = str(MY_HF_HOME)

In [13]:
from huggingface_hub import snapshot_download, try_to_load_from_cache
from typing import List, Optional
import time


def get_cache_dir() -> Path:
    """Get the cache directory."""
    cache_dir = Path(MY_HF_HOME) / "hub" if MY_HF_HOME else Path.home() / ".cache/huggingface/hub"
    Path.mkdir(cache_dir, parents=True, exist_ok=True)
    return cache_dir


def get_model_dir_name(model: str) -> Path:
    """Get the model directory."""
    return f"models--{model.replace('/', '--')}"


def check_model_in_cache(model: str) -> bool:
    """Check if a model's config.json exists in cache."""
    try:
        config_file = try_to_load_from_cache(
            repo_id=model,
            filename="config.json",
        )
        return config_file is not None
    except Exception:
        return False


def download_language_model(model: str, token: Optional[str] = None, force_download: bool = False) -> None:
    """Download a language model if not already in cache.

    Args:
        model: The model identifier (e.g., 'Qwen/Qwen2.5-1.5B')
        token: Optional Hugging Face token for gated models
    """
    print(f"\nChecking model: {model}")

    if not force_download and check_model_in_cache(model):
        print(f"✓ Model {model} already in cache")
        return

    print(f"Downloading {model}...")
    try:
        snapshot_download(
            repo_id=model,
            token=token,
            # _use_symlinks=False
        )
        # sleep for 1 second to avoid rate limit
        time.sleep(1)
        print(f"✓ {model} : Downloaded")
    except Exception as e:
        if "401" in str(e):
            print(f"✗ {model} : Error: Model requires authentication. Please set a correct HF_TOKEN.")
        else:
            print(f"✗ {model} : Error during downloading: {str(e)}")


def list_available_models() -> List[str]:
    """List folders available in cache. Please note that this is not the same as the models available in the Hugging Face Hub."""
    cache_dir = get_cache_dir()
    print(f"Loading models from cache directory: {cache_dir}")
    if not cache_dir.exists():
        return []

    found_models = []
    for m in MODELS_TO_DOWNLOAD:
        config_file = try_to_load_from_cache(
            repo_id=m,
            filename="config.json",
        )
        if config_file is None:
            print(f"✗ {m} : Error: `config.json` of model {m} not found in cache {cache_dir}")
        else:
            # try to load the model with AutoModelForCausalLM
            try:
                from transformers import AutoModelForCausalLM

                AutoModelForCausalLM.from_pretrained(
                    m,
                    use_cache=False,  # KV cache is not needed
                    # trust_remote_code=False,  # avoid issues with remote code
                    local_files_only=True,  # check integrity by only loading from cache
                    token=MY_HF_TOKEN,  # needed for gated models
                )
                found_models.append(m)
                print(f"✓ {m} : Successfully loaded model")
            except Exception as e:
                print(f"✗ {m} : Error loading model: {str(e)}")

    # list only the subfolders of the cache directory, not recursively
    subfolders = [f for f in cache_dir.glob("*") if f.is_dir()]
    print("-" * 32)
    print(f"Cache directory: {cache_dir} has {len(subfolders)} folders.")
    print("Please note their names are not the same as the model names, as per the Hugging Face Hub naming convention")
    print("Found folders in cache are:")
    for f in subfolders:
        print(f"- {f}")

    return found_models

## Double-check Available Models

First, let's see which models are already in your cache:

In [ ]:
print("Checking models in cache ...")
cached_models = list_available_models()
print("-" * 32)
print("Models found currently in cache:")
if cached_models:
    for model in cached_models:
        print(f"- {model}")
else:
    print("No models found in cache")

## Set Hugging Face Token (Optional)

Some models (like Llama) require authentication. If you need to download gated models, set your Hugging Face token here:

In [20]:
# Uncomment and set your token if needed:
# os.environ["HF_TOKEN"] = "your_token_here"  # Get your token from: https://huggingface.co/settings/tokens

## Download Models

Now let's download any missing models:

In [ ]:
token = os.environ.get("HF_TOKEN")

for model in MODELS_TO_DOWNLOAD:
    download_language_model(model, token, force_download=False)

## Final Cache Status

Here are all the models now available in your cache:

In [ ]:
print("Final cache contents:")
cached_models = list_available_models()
print("-" * 32)
print("Models found currently in cache:")
if cached_models:
    for model in cached_models:
        print(f"- {model}")
else:
    print("No models found in cache")

print(f"\nCache directory: {get_cache_dir()}")
print(f"HF_HOME: {os.environ.get('HF_HOME')}")
print("\nTo use these models on an air-gapped system:")
print(f"1. Copy the entire contents of HF_HOME folder to a portable storage device: {MY_HF_HOME} ")
print(
    "2. On the air-gapped system, create the directory path for HF_HOME and set the HF_HOME environment variable (or use default cache directory)"
)
print("3. Copy the contents from your portable storage to the HF_HOME location on the air-gapped system")